In [37]:
import numpy as np
import pandas as pd
from itertools import repeat

from data_loader import BooksDataLoader

In [2]:
books_data_loader = BooksDataLoader()

In [3]:
readers_data = books_data_loader.get_readers_data()

In [4]:
genre_similarity = books_data_loader.get_genre_similarity_matrix()

In [5]:
books_data = books_data_loader.get_books_data()

In [6]:
books_meta_data = books_data_loader.get_books_meta_data()

In [7]:
selected_user = 5

In [8]:
books_reviewed_by_user=set(readers_data[readers_data.user_id == selected_user].book_id.values)

In [9]:
len(books_reviewed_by_user)

100

In [10]:
reviewed_books = genre_similarity[genre_similarity.index.isin(books_reviewed_by_user)].index.values

In [11]:
len(reviewed_books)

5

In [12]:
reviewed_books

array([1347, 1339, 3744, 5766, 4380])

In [31]:
reviewed_books_sub = books_data.loc[books_data.book_id.isin(reviewed_books), ['book_id','original_title', 'authors', 'average_rating']].set_index('book_id')

In [41]:
reviewed_books_sub

,original_title,authors,average_rating
book_id,,,
1339,The Mist,Stephen King,3.92
1347,Revival,Stephen King,3.75
3744,A Drink Before the War,Dennis Lehane,3.96
4380,Fractured,Karin Slaughter,4.18
5766,The Given Day,Dennis Lehane,4.03


In [52]:
pd.set_option('max_colwidth', 200)

In [53]:
display(books_meta_data.loc[books_meta_data.book_id.isin(reviewed_books), ['book_id', 'genre']].set_index('book_id').join(reviewed_books_sub))

,genre,original_title,authors,average_rating
book_id,,,,
1347,Fiction / Horror Fiction / Literary Fiction / Thrillers / Suspense,Revival,Stephen King,3.75
1339,Fiction / Horror Fiction / Thrillers / Suspense,The Mist,Stephen King,3.92
3744,Fiction / Crime Fiction / Mystery & Detective / Hard-Boiled Fiction / Mystery & Detective / Private Investigators,A Drink Before the War,Dennis Lehane,3.96
5766,Fiction / Historical / General Fiction / Literary Fiction / Thrillers / Suspense,The Given Day,Dennis Lehane,4.03
4380,Fiction / Mystery & Detective / Police Procedural Fiction / Thrillers / General Fiction / Thrillers / Suspense,Fractured,Karin Slaughter,4.18


In [18]:
genre_similarity[genre_similarity.index == 54]['1553']

book_id
54    0.235702
Name: 1553, dtype: float64

In [19]:
from functools import partial

recommended_books = set()

def filter_book(book, col):
    val = genre_similarity.loc[genre_similarity.index == int(book), col].values[0]
    if val >= 0.5:
        return True
    else:
        return False

for book in reviewed_books:

    similar_books = set(filter(partial(filter_book, book), genre_similarity.columns.values))
    recommended_books.update(similar_books)

recommended_books = recommended_books - set(reviewed_books)

In [57]:
recommended_books_sub = books_data.loc[books_data.book_id.isin(recommended_books), ['book_id', 'original_title', 'average_rating', 'authors']].nlargest(10, 'average_rating').set_index('book_id')

In [60]:
recommended_books_sub

,original_title,average_rating,authors
book_id,,,
1654,Cold Days,4.51,Jim Butcher
1630,Turn Coat,4.45,Jim Butcher
1450,Small Favor,4.43,Jim Butcher
1394,White Night,4.41,Jim Butcher
6585,"Survivor in Death (In Death, #20)",4.38,J.D. Robb
2830,Consent to Kill,4.36,Vince Flynn
6763,"Innocent in Death (In Death, #24)",4.36,J.D. Robb
3214,Pursuit of Honor,4.35,Vince Flynn
3786,Protect and Defend,4.34,Vince Flynn


In [62]:
books_meta_data.loc[books_meta_data.book_id.isin(recommended_books_sub.index.values), ['genre', 'book_id']].set_index('book_id').join(recommended_books_sub)

,genre,original_title,average_rating,authors
book_id,,,,
6585,Fiction / Mystery & Detective / Police Procedural Fiction / Mystery & Detective / Women Sleuths Fiction / Romance / Suspense,"Survivor in Death (In Death, #20)",4.38,J.D. Robb
6763,Fiction / Mystery & Detective / Police Procedural Fiction / Mystery & Detective / Women Sleuths Fiction / Romance / Suspense,"Innocent in Death (In Death, #24)",4.36,J.D. Robb
1394,Fiction / Fantasy / Paranormal Fiction / Fantasy / Urban Fiction / Mystery & Detective / Private Investigators,White Night,4.41,Jim Butcher
1450,Fiction / Fantasy / Paranormal Fiction / Fantasy / Urban Fiction / Mystery & Detective / Private Investigators,Small Favor,4.43,Jim Butcher
1630,Fiction / Fantasy / Paranormal Fiction / Fantasy / Urban Fiction / Mystery & Detective / Private Investigators,Turn Coat,4.45,Jim Butcher
1654,Fiction / Fantasy / Paranormal Fiction / Fantasy / Urban Fiction / Mystery & Detective / Private Investigators,Cold Days,4.51,Jim Butcher
2830,Fiction / Thrillers / Espionage Fiction / Thrillers / Political Fiction / Thrillers / Suspense,Consent to Kill,4.36,Vince Flynn
3214,Fiction / Thrillers / Espionage Fiction / Thrillers / Political Fiction / Thrillers / Suspense,Pursuit of Honor,4.35,Vince Flynn
3786,Fiction / Thrillers / Espionage Fiction / Thrillers / General Fiction / Thrillers / Political Fiction / Thrillers / Suspense,Protect and Defend,4.34,Vince Flynn
